In [121]:
from Crypto.Cipher import AES
from Crypto import Random
import Crypto.Util.Padding as p
from base64 import b64encode, b64decode
from itertools import chain
import binascii

In [10]:
def get_key(b):
    key = Random.get_random_bytes(b)
    print("Generated key: ", key)
    return key
key = get_key(32)

Generated key:  b'\xfc-\x8f\xbb/j\xde\xe4\tv\x11\nxj\xd2\xe9[\xc6\xe6D\x98(\x06\xba+\xdb\xa3\xed\xde\x020\xa3'


In [14]:
def get_ciphertext(key, text):
    iv = Random.get_random_bytes(16)
    print(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(p.pad(text, AES.block_size))
    return cipher.iv, ciphertext
text = b"This is a ciphertext for AES"
IV, CT = get_ciphertext(key, text)
#IV = b64encode(IV).decode('utf-8')
#CT = b64encode(CT).decode('utf-8')

b'\xf6\xa8\xc1\x96\xc3\x06p\xab\x12#\xc8\xdc\xe9\xe9@C'


In [15]:
print(f"IV: {IV}\nCT:{CT}")

IV: b'\xf6\xa8\xc1\x96\xc3\x06p\xab\x12#\xc8\xdc\xe9\xe9@C'
CT:b'\xa1\xf7a\xeb0M\xda\xef\xedQ\x985Cw4\x9auM\xf9\xfa\x8fu\xdb\xae/4\rg@]Q\x8c'


In [13]:
def check_pad(key, IV, CT):
    cipher = AES.new(key, AES.MODE_CBC, IV)
    PT = cipher.decrypt(CT)
    print(PT)
    try:
        PT = p.unpad(PT, AES.block_size)
        print(PT)
    except ValueError: 
        "Invalid Padding"
        return False
    return True
check = check_pad(key, IV, CT)

b'This is a ciphertext for AES\x04\x04\x04\x04'
b'This is a ciphertext for AES'


In [427]:
BLOCK_SIZE = 16
class AES_CBC_MODE(object):
    
    def __init__(self, key = Random.get_random_bytes(32)):
        
        self.key = key
        self.iv = Random.get_random_bytes(16)
        self.cbc = AES.new(self.key, AES.MODE_CBC, self.iv)
        #self.cbc2 = AES.new(self.key, AES.MODE_CBC, self.iv)
        
    def padding(self, PT):
        pad_len = BLOCK_SIZE - (len(PT) % BLOCK_SIZE)
        padding = bytes([pad_len]) * pad_len
        return PT + padding
    
    def remove_padding(self, PT):
        
        expected_pad = PT[-1]
        
        for i in range(1, expected_pad):
            #print(f"\nbyte: {PT[-i-1]}, expected: {expected_pad}")
            if not PT[-i-1] == expected_pad:
                raise ValueError("Padding Error")
        
        new_PT = PT[:-expected_pad]
        return new_PT
    
    def encrypt(self, PT):
        
        padded_PT = self.padding(PT)
        CT = self.cbc.encrypt(padded_PT)
        CT = self.iv + CT
        return CT
    
    def decrypt(self, CT):
        
        aes = AES.new(self.key, AES.MODE_CBC, self.iv)
        #CT = b64decode(CT)
        #print(CT)
        padded_PT = aes.decrypt(CT[BLOCK_SIZE:])
        PT = self.remove_padding(padded_PT)
            
        return PT
    
    def check_padding(self, CT):
        
        try:
            self.decrypt(CT)
        except:
            return False
        return True
    
cipher = AES_CBC_MODE()

In [428]:
CT = cipher.encrypt(b"please just give me a pass")
print(CT)

b'\xd0\xd2Bs\x8e(\xc6$X}\x8a\xdeU\xa5c3y\xe8\x9a>f&\x9fg|\xf8\xf4\xdel*.\x95~\xc9\xf1\xc7\x94V\x868\xc6\xda\xe2n\xcd\x19&\xab'


In [429]:
PT = cipher.decrypt(CT)
print(PT)


byte: 6, expected: 6

byte: 6, expected: 6

byte: 6, expected: 6

byte: 115, expected: 6

byte: 112, expected: 6
b'please just give me a pass'


In [430]:
cipher.check_padding(CT)


byte: 6, expected: 6

byte: 6, expected: 6

byte: 6, expected: 6

byte: 115, expected: 6

byte: 112, expected: 6


True

### ATTTACKKKKKKKKKKKK!

In [431]:
AES_cbc = AES_CBC_MODE()
def split_data(data):
    blockArr = []
    data_len = len(data)
    #print(data_len, BLOCK_SIZE)
    
    for i in range(int(data_len / BLOCK_SIZE)):
        blockArr.append(data[i * BLOCK_SIZE : (i + 1) * BLOCK_SIZE])
    return blockArr

In [432]:
def get_bytes_in_block(CT):
    
    CT_blocks = bytearray(b64decode(CT))
    print(CT_blocks)
    CT_blocks = split_data(CT_blocks)
    print(CT_blocks)
    
    #manip_iv = bytearray([i for i in CT_blocks[0]])
    manipulated_bytes = bytearray([b for b in CT_blocks[0]])
    #PT_byte = 0
    PT_bytes = bytearray([0] * BLOCK_SIZE)
    
    for padding in range(1, BLOCK_SIZE + 1):
        pad_iv = [padding ^ _ for _ in PT_bytes]
        #pad_iv = bytearray([[0] * (BLOCK_SIZE - padding)] + [[padding + 1] * padding])
    #pad_iv = bytearray([[0] * (BLOCK_SIZE - 1)] + [1])
        for i in range(0, 256):
            #manip_iv[15] = i
            #pad_iv[-padding] = i
            pad_iv[-1] = i
            test_iv = bytes(pad_iv)

            #print("decrypt: ", b64encode(bytes(bytearray(CT_blocks[1])) + i.to_bytes(1, byteorder='big')))
            try:
                print("IV: ", hex(test_iv[-1]))
                print("CT: ", CT_blocks[0][-1])
                #print("decrypt: ", b64encode(bytes(bytearray(CT_blocks[1])) + i.to_bytes(1, byteorder='big')))
                AES_cbc.decrypt(b64encode(CT_blocks[0]), test_iv)
                #PT_byte = byte ^ 0x01 ^ CT_blocks[0][15]
                #print("PT_byte: ",PT_byte)

                break
            except:
                pass

        print(f"padding: {padding}, i: {hex(i)}\n-----")
        manipulated_bytes[-padding] = padding ^ i
        print(manipulated_bytes)
                
    return manipulated_bytes



In [294]:
#print(CT)
#print(get_bytes_in_block(CT))

b'6HRgyeB+N7yiAMU0GhXtGKl8zj99YqirESujpIgR8II='
bytearray(b'\xe8t`\xc9\xe0~7\xbc\xa2\x00\xc54\x1a\x15\xed\x18\xa9|\xce?}b\xa8\xab\x11+\xa3\xa4\x88\x11\xf0\x82')
[bytearray(b'\xe8t`\xc9\xe0~7\xbc\xa2\x00\xc54\x1a\x15\xed\x18'), bytearray(b'\xa9|\xce?}b\xa8\xab\x11+\xa3\xa4\x88\x11\xf0\x82')]
IV:  0x0
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x81')
IV:  0x1
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x80')
IV:  0x2
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x83')
IV:  0x3
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x82')
IV:  0x4
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x85')
IV:  0x5
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x84')
IV:  0x6
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x87')
IV:  0x7
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x86')
IV:  0x8
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x89')
IV:  0x9
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\xde\x97\xd7\x88')
IV:  0xa
CT:  24
bytearray(b'qvx\x05^(-:F=@\x91\

IV:  0xf
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x8e')
IV:  0x10
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x91')
IV:  0x11
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x90')
IV:  0x12
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x93')
IV:  0x13
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x92')
IV:  0x14
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x95')
IV:  0x15
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x94')
IV:  0x16
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x97')
IV:  0x17
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x96')
IV:  0x18
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x99')
IV:  0x19
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x98')
IV:  0x1a
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x9b')
IV:  0x1b
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x9a')
IV:  0x1c
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x9d')
IV:  0x1d
CT:  24
bytearray(b'ur|\x01Z,)>B9D\x95\xda\x93\xd3\x9

bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe0')
IV:  0x62
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe3')
IV:  0x63
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe2')
IV:  0x64
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe5')
IV:  0x65
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe4')
IV:  0x66
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe7')
IV:  0x67
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe6')
IV:  0x68
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe9')
IV:  0x69
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xe8')
IV:  0x6a
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xeb')
IV:  0x6b
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xea')
IV:  0x6c
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xed')
IV:  0x6d
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xec')
IV:  0x6e
CT:  24
bytearray(b'x\x7fq\x0cW!$3O4I\x98\xd7\x9e\xde\xef')
IV:  0x6f
CT:  24
bytearray(b'x\x7fq\x

IV:  0x1f
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\x9e')
IV:  0x20
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa1')
IV:  0x21
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa0')
IV:  0x22
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa3')
IV:  0x23
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa2')
IV:  0x24
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa5')
IV:  0x25
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa4')
IV:  0x26
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa7')
IV:  0x27
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa6')
IV:  0x28
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa9')
IV:  0x29
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xa8')
IV:  0x2a
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xab')
IV:  0x2b
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xaa')
IV:  0x2c
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xad')
IV:  0x2d
CT:  24
bytearray(b'}zt\tR$!6J1L\x9d\xd2\x9b\xdb\xac')
IV:  0x2e
CT:  24
bytearr

In [290]:
print(manipulated_bytes)

NameError: name 'manipulated_bytes' is not defined

In [444]:
# AES128 implementation

BLOCK_SIZE = AES.block_size

_random_gen = Random.new()
key = _random_gen.read(16)


def _add_padding(msg):
    pad_len = BLOCK_SIZE - (len(msg) % BLOCK_SIZE)
    padding = bytes([pad_len]) * pad_len
    return msg + padding


def _remove_padding(data):
    pad_len = data[-1]

    if pad_len < 1 or pad_len > BLOCK_SIZE:
        return None
    for i in range(1, pad_len):
        if data[-i-1] != pad_len:
            return None
    return data[:-pad_len]


def AES_encrypt(data):
    iv = _random_gen.read(AES.block_size)
    cipher = AES.new(_key, AES.MODE_CBC, iv)
    CT = iv + cipher.encrypt(_add_padding(data))
    return CT


def AES_decrypt(data):
    iv = data[:BLOCK_SIZE]
    cipher = AES.new(_key, AES.MODE_CBC, iv)
    return _remove_padding(cipher.decrypt(data[BLOCK_SIZE:]))


def check_padding(data):
    return AES_decrypt(data) is not None

In [453]:
KEY_LENGTH = 16

split_string = lambda x, n: [x[i:i+n] for i in range(0, len(x), n)]

def full_attack(ciphertext):
    
    # Final result, checked with oracle
    guessed_clear = b''
    
    
    blocks = split_string(ciphertext, BLOCK_SIZE)
    print(blocks)
    
    for block_n in range( len(blocks) -1, 0, -1 ):
        #if block_n == 1:
        #    manipulated_CT = cipher.iv + blocks[block_n - 1]
        #else:
        #    manipulated_CT = blocks[block_n - 2] + blocks[block_n - 1]
        manipulated_CT = blocks[block_n - 1] + blocks[block_n]
        print(block_n)
        decoded_bytes = b'?' * BLOCK_SIZE

        # Do attack for each block in Ciphertext
        for byte in range( BLOCK_SIZE - 1, -1, -1 ):
            print("BYTE NUMBER: ", byte)
            new_pad_len = BLOCK_SIZE - byte
            
            # setup to hack next byte
            hacked_ciphertext_tail = b''
            for padder_index in range( 1, new_pad_len ):
                hacked_ciphertext_tail += bytearray.fromhex('{:02x}'.format( new_pad_len ^ decoded_bytes[byte + padder_index] ) )
                
            for i in range( 0, 256 ):
                attack_str = bytearray.fromhex( '{:02x}'.format( ( i ^ manipulated_CT[byte] ) ) )
                hacked_ciphertext = manipulated_CT[:byte] + attack_str + hacked_ciphertext_tail + manipulated_CT[byte + 1 + new_pad_len - 1:]

                if(check_padding(hacked_ciphertext)):

                    test = hacked_ciphertext[:byte - 1] + bytearray.fromhex( '{:02x}'.format( ( 1 ^  hacked_ciphertext[byte] ) ) )  + hacked_ciphertext[byte:]
                    if(not check_padding(test)):
                        continue
                        
                    decoded_bytes = decoded_bytes[:byte] + bytearray.fromhex('{:02x}'.format( hacked_ciphertext[byte] ^ new_pad_len ) ) + decoded_bytes[byte + 1:]
                    guessed_clear = bytearray.fromhex('{:02x}'.format( i ^ new_pad_len ) ) + guessed_clear
                    print("GUESSED: ", guessed_clear)
                    break
                    
    return guessed_clear[:-guessed_clear[-1]] #remove padding!

In [454]:
BLOCK_SIZE = 16
PT = b"this is a test."

hacked_PT = attack(encrypt(PT))

[b'\xe6\x15\xfa[\xd6\xe3\x8f\x8d\x9d\xbeZ\x02\xab{\xe5\xc6', b'^R(\x8c\x19\xa6:h>\xb9\x8e\xfc\x0eL\xa9x']
1
BYTE NUMBER:  15
GUESSED:  bytearray(b'\x01')
BYTE NUMBER:  14
GUESSED:  bytearray(b'.\x01')
BYTE NUMBER:  13
GUESSED:  bytearray(b't.\x01')
BYTE NUMBER:  12
GUESSED:  bytearray(b'st.\x01')
BYTE NUMBER:  11
GUESSED:  bytearray(b'est.\x01')
BYTE NUMBER:  10
GUESSED:  bytearray(b'test.\x01')
BYTE NUMBER:  9
GUESSED:  bytearray(b' test.\x01')
BYTE NUMBER:  8
GUESSED:  bytearray(b'a test.\x01')
BYTE NUMBER:  7
GUESSED:  bytearray(b' a test.\x01')
BYTE NUMBER:  6
GUESSED:  bytearray(b's a test.\x01')
BYTE NUMBER:  5
GUESSED:  bytearray(b'is a test.\x01')
BYTE NUMBER:  4
GUESSED:  bytearray(b' is a test.\x01')
BYTE NUMBER:  3
GUESSED:  bytearray(b's is a test.\x01')
BYTE NUMBER:  2
GUESSED:  bytearray(b'is is a test.\x01')
BYTE NUMBER:  1
GUESSED:  bytearray(b'his is a test.\x01')
BYTE NUMBER:  0
GUESSED:  bytearray(b'this is a test.\x01')


In [455]:
print(hacked_PT)

bytearray(b'this is a test.')
